In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/synthetic-oversampling-tvaes.csv')

In [3]:
df.head()

,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,clin-reservaCognitiva_idiomas,clin-reservaCognitiva_ocupacion,clin-reservaCognitiva_escolaridad,clin-ansiedad,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,2,2024-02-22T00:00:00,1942-05-08T00:00:00,Rango3,81,12,0,1,3,False,...,84.09,0.14,-20.00,0.00,33.33,37.50,-33.33,-38.46,-7.69,D
1,2,2024-06-26T00:00:00,1948-10-07T00:00:00,Rango3,75,8,0,1,3,True,...,79.55,-0.33,-71.43,12.50,0.00,0.00,0.00,-20.00,-20.00,D
2,2,2024-05-14T00:00:00,1952-05-08T00:00:00,Rango2,72,11,0,0,4,False,...,88.64,-0.60,-14.29,75.00,25.00,-20.00,0.00,-16.67,-16.67,D
3,2,2024-04-10T00:00:00,1964-01-27T00:00:00,Rango1,60,18,2,1,4,False,...,90.91,0.60,60.00,16.67,0.00,50.00,-20.00,-50.00,-37.50,D
4,2,2024-05-14T00:00:00,1959-09-16T00:00:00,Rango1,64,13,0,1,4,False,...,86.36,-0.71,0.00,66.67,0.00,-33.33,0.00,-10.00,-10.00,D


In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((5196, 77), (1300, 77))

In [5]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v4'
    )

In [6]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 2 * 3600,
    auto_stack = True,
    # excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       8.28 GB / 15.94 GB (51.9%)
Disk Space Avail:   81.28 GB / 446.36 GB (18.2%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and

(_ray_fit pid=23528) [1000]	valid_set's binary_logloss: 0.358346


(_dystack pid=14912) 	0.9053	 = Validation score   (roc_auc)
(_dystack pid=14912) 	8.32s	 = Training   runtime
(_dystack pid=14912) 	0.42s	 = Validation runtime
(_dystack pid=14912) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 980.76s of the 1578.11s of remaining time.
(_dystack pid=14912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.43%)
(_ray_fit pid=1124) No improvement since epoch 0: early stopping
(_dystack pid=14912) 	0.8657	 = Validation score   (roc_auc)
(_dystack pid=14912) 	17.57s	 = Training   runtime
(_dystack pid=14912) 	0.28s	 = Validation runtime
(_dystack pid=14912) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 958.83s of the 1556.18s of remaining time.
(_dystack pid=14912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=2.74%)
(_dystack pid=14912) 	0.9032	 = Validatio

(_ray_fit pid=9124) [1000]	valid_set's binary_logloss: 0.430241 [repeated 8x across cluster]
(_ray_fit pid=14692) [3000]	valid_set's binary_logloss: 0.371764 [repeated 21x across cluster]


(_dystack pid=14912) 	0.908	 = Validation score   (roc_auc)
(_dystack pid=14912) 	9.23s	 = Training   runtime
(_dystack pid=14912) 	1.31s	 = Validation runtime
(_dystack pid=14912) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 777.17s of the 1374.52s of remaining time.
(_dystack pid=14912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.21%)
(_dystack pid=14912) 	0.8771	 = Validation score   (roc_auc)
(_dystack pid=14912) 	23.6s	 = Training   runtime
(_dystack pid=14912) 	0.21s	 = Validation runtime
(_dystack pid=14912) Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 749.38s of the 1346.73s of remaining time.
(_dystack pid=14912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=3.67%)
(_dystack pid=14912) 	0.8955	 = Validation score   (roc_auc)
(_dystack pid=14912) 	23.88s	 = Training   runtim

(_ray_fit pid=16228) [1000]	valid_set's binary_logloss: 0.445681 [repeated 2x across cluster]
(_ray_fit pid=16228) [2000]	valid_set's binary_logloss: 0.434876 [repeated 8x across cluster]
(_ray_fit pid=4880) [3000]	valid_set's binary_logloss: 0.354462 [repeated 8x across cluster]


(_dystack pid=14912) 	0.9017	 = Validation score   (roc_auc)
(_dystack pid=14912) 	21.85s	 = Training   runtime
(_dystack pid=14912) 	2.37s	 = Validation runtime
(_dystack pid=14912) Fitting model: RandomForest_r39_BAG_L1 ... Training model for up to 71.82s of the 669.18s of remaining time.
(_dystack pid=14912) 	0.8738	 = Validation score   (roc_auc)
(_dystack pid=14912) 	3.39s	 = Training   runtime
(_dystack pid=14912) 	0.21s	 = Validation runtime
(_dystack pid=14912) Fitting model: CatBoost_r167_BAG_L1 ... Training model for up to 68.16s of the 665.51s of remaining time.
(_dystack pid=14912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.97%)
(_dystack pid=14912) 	0.9028	 = Validation score   (roc_auc)
(_dystack pid=14912) 	46.75s	 = Training   runtime
(_dystack pid=14912) 	0.09s	 = Validation runtime
(_dystack pid=14912) Fitting model: NeuralNetFastAI_r95_BAG_L1 ... Training model for up to 17.24s of th

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                 WeightedEnsemble_L2   0.919472     roc_auc       2.756765  236.724258                0.000000           0.401107            2      False        111
1                 WeightedEnsemble_L3   0.919195     roc_auc       7.100462  541.890947                0.000000           0.399814            3      False        220
2                 CatBoost_r12_BAG_L2   0.918341     roc_auc       6.313657  413.874296                0.076711          40.364775            2      False        205
3                 CatBoost_r13_BAG_L2   0.918335     roc_auc       6.320304  455.220077                0.083358          81.710555            2      False        134
4                CatBoost_r177_BAG_L2   0.918316     roc_auc       6.295930  389.046456                0.058

c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [8]:
auxiliary_metrics = ['accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [9]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9174161257527176
Evaluations on test data:
{
    "roc_auc": 0.9174161257527176,
    "accuracy": 0.8476923076923077,
    "balanced_accuracy": 0.8482644541923497,
    "mcc": 0.6961133153549074,
    "f1": 0.8455538221528861,
    "precision": 0.8274809160305343,
    "recall": 0.8644338118022329
}


{'roc_auc': 0.9174161257527176,
 'accuracy': 0.8476923076923077,
 'balanced_accuracy': 0.8482644541923497,
 'mcc': 0.6961133153549074,
 'f1': 0.8455538221528861,
 'precision': 0.8274809160305343,
 'recall': 0.8644338118022329}

In [10]:
predictor.model_best

'WeightedEnsemble_L2_FULL'

In [11]:
predictors = predictor.leaderboard(
    df_test,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [12]:
predictors[['model', 'roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'num_features']]

,model,roc_auc,accuracy,balanced_accuracy,f1,precision,recall,num_features
0,LightGBM_r188_BAG_L2_FULL,0.918639,0.842308,0.843391,0.842429,0.813056,0.874003,105
1,NeuralNetTorch_r158_BAG_L2_FULL,0.918054,0.840000,0.838873,0.829508,0.853288,0.807018,105
2,LightGBM_r96_BAG_L2_FULL,0.917781,0.843846,0.844768,0.843243,0.817365,0.870813,105
3,LightGBM_r196_BAG_L2_FULL,0.917764,0.845385,0.846581,0.845977,0.814159,0.880383,105
4,LightGBM_r30_BAG_L2_FULL,0.917722,0.846154,0.847215,0.846154,0.817236,0.877193,105
...,...,...,...,...,...,...,...,...
435,NeuralNetFastAI_r191_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
436,NeuralNetFastAI_r127_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
437,NeuralNetFastAI_r88_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
438,NeuralNetFastAI_r95_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82


In [13]:
predictors.to_excel(
    '../../data/tlp/predictors-tvaes-oversampling.xlsx',
    index=False
)

In [14]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido']
Computing feature importance via permutation shuffling for 75 features using 5000 rows with 5 shuffle sets...
	235.53s	= Expected runtime (47.11s per shuffle set)
	141.95s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
eval-TLP-Stroop-palabrasColor-PD,0.010526,0.000507,6.420991e-07,5,0.011569,0.009483
demo-fechaNacimiento,0.006954,0.000478,2.667325e-06,5,0.007939,0.005969
eval-TLP-CubCorsi-total-PD,0.006881,0.000340,7.098903e-07,5,0.007580,0.006181
eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,0.004070,0.000500,2.684887e-05,5,0.005100,0.003040
eval-TLP-Tavec-4_RI_B-PD,0.003078,0.000334,1.632316e-05,5,0.003765,0.002391
...,...,...,...,...,...,...
eval-TLP-Tavec-3_RI_AT-PD,0.000090,0.000007,4.466569e-06,5,0.000104,0.000075
eval-TLP-Tavec-11_RCl_LP-PD,0.000081,0.000040,5.337618e-03,5,0.000163,-0.000002
ques-MMSE-orientacion-PD,0.000063,0.000027,3.025087e-03,5,0.000118,0.000008
clin-cardiopatiaIsquemica,0.000060,0.000010,8.205060e-05,5,0.000080,0.000040
